# 연관분석(Association Analysis)
상품이나 서비스를 구매하는 등 일련의 거래나 사건 안에 존재하는 **항목 간의 일정한 연관 규칙을 발견하는 분석이다.**

**장바구니 분석(Basket Analysis)**이라고도 하나, 엄밀히 짚고 넘어가면 장바구니 분석은 유통업에서 부르는 용어이고 연관분석의 한 종류라고 볼 수 있다.

연관분석 알고리즘으로는 `Apriori`, `FP-Growth`, `FPV`가 있다.

<br>

## 연관 규칙의 형태
'A를 구매했을 때, B 또한 구매할 것이다' 와 같이 `if A, then B`의 패턴으로 이루어져있다.

예시)
- 맥주를 구매하는 사람은 기저귀를 함께 구매한다.
- 샌드위치를 먹는 사람은 탄산수를 함께 마신다.
- 사탕을 구매하는 사람은 키보드를 함께 구매한다.

<br>

## 연관분석 측정지표
대표적인 연관규칙 측정지표로는 `지지도`, `신뢰도`, `향상도`가 있으며, 산업의 특성에 따라 중요한 지표가 달라진다.

<br>

### 1. 지지도(Support)
A → B 의 지지도는 전체 거래중에서 **A와 B가 동시에 판매되는 거래의 비율**을 의미한다. 모든 경우의 수를 분석하는 것은 불필요한 연산을 증가시키므오로 `최소지지도`라는 것을 설정해 이 값이 임계값을 넘는 품목에 대한 규칙을 도출한다.
**1에 가까울수록 연관성이 높다**고 할 수 있다.

- $Support = A와 B를 모두 포함하는 거래 수 / 전체 거래 수$

<br>

### 2. 신뢰도(Confidence)
A → B의 신뢰도는 **A의 거래 중에서 B가 포함된 거래의 비율**로, 상품 간에 존재하는 연관성의 정도를 측정하는 데 사용된다.
**A를 구매했을 때 B도 구매할 조건부 확률**이라고도 할 수 있다. 1에 가까울수록 연관성이 높다.

- $Confidence = A와 B를 모두 포함하는 거래 수 / A를 포함하는 거래 수$
- $Confidence = 지지도 / P(A)$

<br>

### 3. 향상도(Lift)
A → B의 향상도는 A를 구매하지 않았을 때 B를 구매할 확률 대비 A를 구매했을 때 B를 구매 할 **구매확률의 증가 비율**을 의미한다. **향상도가 1이면 두 품목은 독립**이다(관련이 없다). 1보다 작으면 상관관계를 가지며 A를 구매하면 B를 구매하지 않을 확률이 구매할 확률보다 큼을 의미한다. 향상도가 1보다 크면 두 품목은 양의 상관관계를 가지고 임의로 B를 구매할 확률보다 A를 구매했을 때 B를 구매할 확률이 더 큼을 의미한다.

- $Lift = 신뢰도 / P(B)$

<br>

- 향상도 > 1 : 우연적 기회보다 높은 확률
- 향상도 = 1 : 독립
- 향상도 < 1 : 우연적 기회보다 낮은 확률

<br>

## 장점
- 탐색적인 기법으로 분석 결과를 쉽게 이해할 수 있다.
- 강력한 비목적성 분석기법으로 유용하게 활용이 가능하다.
- 분석을 위한 계산이 간단하다.

## 단점
- 품목수가 증가하면 분석에 필요한 계산은 기하급수적으로 늘어난다. -> 유사 품목을 재범주화 혹은 신뢰도 하한을 새롭게 정의하고 진행
- 너무 세분화된 품목이 있을 경우 규칙 발견시 의미없는 분석이 될 수 있다.
- 거래량이 적은 품목은 규칙 발견시 제외되기 쉽다.

<br>

## 절차
- 최소 지지도 보다 큰 집합만을 대상으로 높은 지지도를 갖는 품목 집합을 찾는다.
- 5%에서 시작하여 규칙이 충분히 도출되는지를 보고 다양하게 조절하여 시도

In [1]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('/content/chipotle.tsv', engine='python', delimiter='\t')

In [4]:
print(df.shape)
df.head()

(4622, 5)


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [21]:
df['item_name'] = [[i] for i in df['item_name']]

In [23]:
item_df = df.groupby('order_id')['item_name'].sum().reset_index(name='item_list')
item_df

,order_id,item_list
0,1,"[Chips and Fresh Tomato Salsa, Izze, Nantucket..."
1,2,[Chicken Bowl]
2,3,"[Chicken Bowl, Side of Chips]"
3,4,"[Steak Burrito, Steak Soft Tacos]"
4,5,"[Steak Burrito, Chips and Guacamole]"
...,...,...
1829,1830,"[Steak Burrito, Veggie Burrito]"
1830,1831,"[Carnitas Bowl, Chips, Bottled Water]"
1831,1832,"[Chicken Soft Tacos, Chips and Guacamole]"
1832,1833,"[Steak Burrito, Steak Burrito]"


In [27]:
item_df['item_list'] = [list(set(i)) for i in item_df['item_list']]

In [28]:
item_df.sample(10)

,order_id,item_list
1550,1551,"[Chicken Bowl, Bottled Water]"
1756,1757,"[Chicken Burrito, Chips and Guacamole, Bottled..."
834,835,"[Chicken Bowl, Chips and Guacamole]"
1460,1461,"[Chicken Bowl, Chicken Salad Bowl]"
877,878,"[Chicken Bowl, Chips and Guacamole]"
1799,1800,"[Chicken Burrito, Chips and Guacamole, 6 Pack ..."
1285,1286,"[Chicken Bowl, Chips and Guacamole]"
13,14,"[Canned Soda, Carnitas Burrito]"
515,516,"[Chicken Burrito, Side of Chips]"
529,530,"[Chips, Chicken Bowl, Barbacoa Burrito]"


## MLXTEND

In [29]:
from mlxtend.preprocessing import TransactionEncoder 
from mlxtend.frequent_patterns import apriori, association_rules

In [30]:
te = TransactionEncoder()

In [32]:
te_array = te.fit_transform(item_df['item_list'])
te_array

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [36]:
te_df = pd.DataFrame(te_array, columns=te.columns_)
te_df.head()

,6 Pack Soft Drink,Barbacoa Bowl,Barbacoa Burrito,Barbacoa Crispy Tacos,Barbacoa Salad Bowl,Barbacoa Soft Tacos,Bottled Water,Bowl,Burrito,Canned Soda,...,Steak Crispy Tacos,Steak Salad,Steak Salad Bowl,Steak Soft Tacos,Veggie Bowl,Veggie Burrito,Veggie Crispy Tacos,Veggie Salad,Veggie Salad Bowl,Veggie Soft Tacos
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


최소 지지도가 0.05 이상인 규칙 집합

- Bottled Water를 구매할 확률은 0.083969이다.
- Chicken Bowl와 Chips and Guacamole를 같이 구매할 확률은 0.081243이다.

In [39]:
freq_items = apriori(te_df, min_support=0.05, use_colnames=True)
freq_items.sample(10)

,support,itemsets
16,0.062159,"(Chicken Burrito, Chips and Guacamole)"
0,0.083969,(Bottled Water)
5,0.053435,(Chicken Salad Bowl)
4,0.266630,(Chicken Burrito)
3,0.335333,(Chicken Bowl)
2,0.150491,(Canned Soft Drink)
1,0.051254,(Canned Soda)
8,0.059978,(Chips and Fresh Tomato Salsa)
15,0.081243,"(Chicken Bowl, Chips and Guacamole)"
7,0.113413,(Chips)


- lift(향상도)가 1보다 클수록 우연히 일어나지 않았다는 의미다. 아무런 관계가 없을 경우 1로 표시된다.

Canned Soft Drink과 Chicken Bowl이 함께 등장할 확률(지지도)은 0.060523으로 약 6%이다. 이 때 향상도는 1.199328로 Chicken Bowl만 구매할때 보다 Canned Soft Drink와 Chicken Bowl이 함께 구매될 확률이 1.19배 높다는 것을 의미한다.

In [40]:
association_rules(freq_items, metric='confidence', min_threshold=0.05)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
1,(Chicken Bowl),(Canned Soft Drink),0.335333,0.150491,0.060523,0.180488,1.199328,0.010059,1.036604
2,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
3,(Chicken Bowl),(Chips),0.335333,0.113413,0.066521,0.198374,1.749124,0.028490,1.105985
4,(Chicken Bowl),(Chips and Guacamole),0.335333,0.258451,0.081243,0.242276,0.937416,-0.005424,0.978653
5,(Chips and Guacamole),(Chicken Bowl),0.258451,0.335333,0.081243,0.314346,0.937416,-0.005424,0.969392
6,(Chicken Burrito),(Chips and Guacamole),0.266630,0.258451,0.062159,0.233129,0.902022,-0.006752,0.966979
7,(Chips and Guacamole),(Chicken Burrito),0.258451,0.266630,0.062159,0.240506,0.902022,-0.006752,0.965603


### APYORI

In [43]:
# pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=3bd580d4c87e47952584a8df2e4d637f43b670b5ecb2918180778b67b5f2b6a1
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [44]:
from apyori import apriori

In [45]:
item_df.head()

,order_id,item_list
0,1,"[Chips and Fresh Tomato Salsa, Chips and Tomat..."
1,2,[Chicken Bowl]
2,3,"[Chicken Bowl, Side of Chips]"
3,4,"[Steak Soft Tacos, Steak Burrito]"
4,5,"[Chips and Guacamole, Steak Burrito]"


전체 주문에서 5% 이상의 주문 접수가 된 품목 탐색

In [49]:
association_rules = apriori(item_df['item_list'], min_support=0.05, min_confidence=0.0005)
association_result = list(association_rules)

In [53]:
rules1 = pd.DataFrame(association_result)
rules1.head()

,items,support,ordered_statistics
0,(Bottled Water),0.083969,"[((), (Bottled Water), 0.08396946564885496, 1.0)]"
1,(Canned Soda),0.051254,"[((), (Canned Soda), 0.05125408942202835, 1.0)]"
2,(Canned Soft Drink),0.150491,"[((), (Canned Soft Drink), 0.1504907306434024,..."
3,(Chicken Bowl),0.335333,"[((), (Chicken Bowl), 0.33533260632497275, 1.0)]"
4,(Chicken Burrito),0.266630,"[((), (Chicken Burrito), 0.26663031624863687, ..."


In [52]:
print(f'연관분석에서 생성된 규칙의 개수는 {rules1.shape[0]}개이다.')

연관분석에서 생성된 규칙의 개수는 17개이다.


In [58]:
rules1['lhs'] = [i[0][0] for i in rules1.ordered_statistics]  # 좌측항 생성
rules1['rhs'] = [i[0][1] for i in rules1.ordered_statistics]  # 우측항 생성

In [59]:
rules1['confindence'] = [i[0][2] for i in rules1.ordered_statistics]  # 신뢰도 생성
rules1['lift'] = [i[0][3] for i in rules1.ordered_statistics]  # 향상도 생성

In [61]:
rules1['n_item'] = rules1['items'].apply(lambda x : len(x))

In [62]:
rules1.drop(['ordered_statistics', 'items'], axis=1, inplace=True)  # 불필요한 열 삭제

In [65]:
rules1

,support,lhs,rhs,confindence,lift,n_item
0,0.083969,(),(Bottled Water),0.083969,1.0,1
1,0.051254,(),(Canned Soda),0.051254,1.0,1
2,0.150491,(),(Canned Soft Drink),0.150491,1.0,1
3,0.335333,(),(Chicken Bowl),0.335333,1.0,1
4,0.266630,(),(Chicken Burrito),0.266630,1.0,1
5,0.053435,(),(Chicken Salad Bowl),0.053435,1.0,1
6,0.058342,(),(Chicken Soft Tacos),0.058342,1.0,1
7,0.113413,(),(Chips),0.113413,1.0,1
8,0.059978,(),(Chips and Fresh Tomato Salsa),0.059978,1.0,1
9,0.258451,(),(Chips and Guacamole),0.258451,1.0,1


향상도가 1로 나온 이유는 이전 구매 품목(lhs)가 빈칸으로 되어있고 이후 구매 품목(rhs)에는 품목이 존재해서 lhs, rhs가 서로 독립이기 때문이다.

In [67]:
rules1[rules1['n_item'] > 1]

,support,lhs,rhs,confindence,lift,n_item
13,0.060523,(),"(Canned Soft Drink, Chicken Bowl)",0.060523,1.0,2
14,0.066521,(),"(Chips, Chicken Bowl)",0.066521,1.0,2
15,0.081243,(),"(Chicken Bowl, Chips and Guacamole)",0.081243,1.0,2
16,0.062159,(),"(Chicken Burrito, Chips and Guacamole)",0.062159,1.0,2
